### Извлечение информации для каждого пользователя

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import multiprocessing
import pymorphy2
from concurrent.futures import ThreadPoolExecutor
from itertools import chain
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
num_cores = multiprocessing.cpu_count()

## Функция предобработки названия товара:
- Удаляем обычные цифры и различные скобки
- Приведение к нижнему регистру
- Токенизация(извлечение слов из текста)
- Морфологизация слов(приведение к нормальной форме)

In [4]:
morph = pymorphy2.MorphAnalyzer()

# Загрузка стоп-слов заранее, чтобы не загружать их при каждом вызове функции
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9]', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация текста
    tokens = word_tokenize(text)
    
    preprocessed_text = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]
    
    return preprocessed_text

## Чтение данного нам датасета
1. превращаем имена покупателей в уникальные идентификаторы
2. Извлекаю максимум информации из датасета

In [6]:
df = pd.read_csv('../final_ds_finnopolis.csv', names=['Answer', 'Company_id', 'Price', 'Name'])

In [4]:
df

,Answer,Company_id,Price,Name
0,Оперативная память apacer nox [ah4u16g32c28ymb...,1838685550,4399,Блахов Василий Александрович
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,Блахов Василий Александрович
2,Оперативная память kingston fury beast black r...,1838685550,3599,Блахов Василий Александрович
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,Блахов Василий Александрович
4,Газовые упоры капота geely coolray / джили кул...,1197983357,678,Блахов Василий Александрович
...,...,...,...,...
299995,Колбаса вареная ближние горки молочная гост 450 г,1363498391,369,Малименкова Зоя Александровна
299996,Мед берестов а.с. с прополисом 500 г,1363498391,38,Малименкова Зоя Александровна
299997,Чай черный greenfield magic yunnan в пакетиках...,1976872004,131,Малименкова Зоя Александровна
299998,Колбаса останкино сервелат зернистый варено-ко...,1363498391,329,Малименкова Зоя Александровна


In [7]:
labelencoder = LabelEncoder()
df['Name'] = labelencoder.fit_transform(df.Name)

answerList_for_id - список вещей, которые запросил определенный пользователь  
PriceCount_for_id - Сумма, потраченная определенным человеком  
Company_for_id - сколько потратил определенный человек на определенную компанию

In [8]:
answerList_for_id = df.groupby('Name')['Answer'].agg(list)
PriceCount_for_id = df.groupby('Name')['Price'].agg(sum)
Company_for_id = df.groupby(['Name', 'Company_id'])['Price'].sum()

In [9]:
df

,Answer,Company_id,Price,Name
0,Оперативная память apacer nox [ah4u16g32c28ymb...,1838685550,4399,911
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,911
2,Оперативная память kingston fury beast black r...,1838685550,3599,911
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,911
4,Газовые упоры капота geely coolray / джили кул...,1197983357,678,911
...,...,...,...,...
299995,Колбаса вареная ближние горки молочная гост 450 г,1363498391,369,5336
299996,Мед берестов а.с. с прополисом 500 г,1363498391,38,5336
299997,Чай черный greenfield magic yunnan в пакетиках...,1976872004,131,5336
299998,Колбаса останкино сервелат зернистый варено-ко...,1363498391,329,5336


## Найдем уникальные колонки - виды товаров
##### P.S. датасеты теже - из ozon

In [10]:
onlyfiles = ['data_cat/dat2/' + f for f in os.listdir('data_cat/dat2') if os.path.isfile(os.path.join('data_cat/dat2', f))]

In [11]:
def process_dataframe(file):
    df = pd.read_excel(file)
    df['Категория 1 уровня'] = df['Категория 1 уровня'].astype(str)
    return df['Категория 1 уровня'].unique()
    

col = []
pool = multiprocessing.Pool(processes=num_cores)

# Использование map для параллельной обработки данных
col = pool.map(process_dataframe, onlyfiles)

# Закрытие пула процессов
pool.close()
pool.join()
col = list(set(chain.from_iterable(col)))
col.remove('i9410865678000 _x000D_\n')

Добавим эти колонки в датасет

In [12]:
for i in col:
    df[i] = [None] * len(df)

In [13]:
data = df.iloc[:, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]].copy()

Дальше - фишка модели:
- Модель обучается на каждом новом чеке, те она будет все лучше и лучше классифицировать(чтобы не переобучилась стоит низкий learning rate)
- Модель нужна для того чтобы классифицировать название по категориям
- В чем особенность? Благодаря тому, что данная модель - NLP, она способна не только классифицировать, но и искать связи между словами, т.е. с помощью данной модели можно также рекомендовать похожие товары и прочее 
- Достаточно высокая скорость работы и небольшая занимаемая память

In [14]:
num_cores = multiprocessing.cpu_count()

# Создание пула процессов
pool = multiprocessing.Pool(processes=num_cores)

# Применение функции preprocess_text к столбцу 'Answer'
data['Answer'] = pool.map(preprocess_text, data['Answer'])

# Закрытие пула процессов
pool.close()
pool.join()

In [15]:
data_Ans = data.Answer.tolist().copy()

In [41]:
model = Word2Vec.load('word2vec')
model.build_vocab(data_Ans, update=True)
model.alpha = 0.01
# Дообучите модель на новых данных
model.train(data_Ans, total_examples=model.corpus_count, epochs=1)

(1671148, 1851185)

In [55]:
vector1 = model.wv['оперативный']  # Получение векторного представления строки 1
vector2 = model.wv['одежда']  # Получение векторного представления строки 2

# Рассчет косинусного расстояния
model.wv.cosine_similarities(vector1, [vector2])[0]

0.16286726

In [48]:
# def calculate_similarity(args):
#     index, row, col_name = args
#     answer_vectors = [model.wv[word] for word in row['Answer']]
#     col_vector = model.wv[col_name]
#     similarities = model.wv.cosine_similarities(col_vector, answer_vectors)
#     return index, col_name, np.mean(np.maximum(similarities, 0))

# with multiprocessing.Pool(processes=num_cores) as pool:
#     args_list = ((index, row, col_name) for index, row in data.iterrows() for col_name in data.columns if col_name not in ['Answer', 'Name'])
#     results = pool.imap_unordered(calculate_similarity, args_list)

#     for index, col_name, similarity in results:
#         data.at[index, col_name] = similarity

In [ ]:
def calculate_similarity(index, row, col_name):
    answer_vectors = [model.wv[word] for word in row['Answer']]
    col_vector = model.wv[col_name]
    similarities = model.wv.cosine_similarities(col_vector, answer_vectors)
    return index, col_name, np.mean(np.maximum(similarities, 0))

args_list = ((index, row, col_name) for index, row in data.iterrows() for col_name in data.columns if col_name not in ['Answer', 'Name'])

with ThreadPoolExecutor(max_workers=num_cores) as executor:
    results = executor.map(lambda args: calculate_similarity(*args), args_list)

for index, col_name, similarity in results:
    data.at[index, col_name] = similarity

In [49]:
pool.join()
pool.close()

In [63]:
data

,Answer,Name,Товары для дома,Entertainment certificates,Автотовары,ТВ и аудио,Свежие продукты питания,Товары для мам и детей,Одежда,DIY,...,Книги,Красота и здоровье,NewBorn,Спорт и отдых,Ювелирные изделия,Компьютеры и ноутбуки,Продукты питания,Товары для животных,Мебель,Товары повседневного спроса
0,"[оперативный, память, apacer, nox, ah4u16g32c2...",911,0.233985,0.344255,0.301514,0.469033,0.249215,0.3586,0.227517,0.224213,...,0.435718,0.306469,0.258979,0.242221,0.305348,0.476621,0.154391,0.359499,0.294874,0.344474
1,"[видеокарта, kfa2, geforce, rtx, x, black, 35n...",911,0.31453,0.521324,0.27,0.299432,0.286479,0.230729,0.339655,0.357172,...,0.400982,0.335735,0.232374,0.315515,0.425404,0.545673,0.294164,0.365769,0.411719,0.258101
2,"[оперативный, память, kingston, fury, beast, b...",911,0.230929,0.381401,0.299406,0.44339,0.254926,0.357999,0.232324,0.264696,...,0.44126,0.316416,0.26616,0.256677,0.30854,0.458889,0.178846,0.374134,0.299524,0.347242
3,"[свеча, зажигание, ngk, cr7hsa, шт]",911,0.277128,0.370164,0.334299,0.427887,0.192633,0.25499,0.13752,0.284665,...,0.40198,0.240009,0.227765,0.203162,0.378024,0.295506,0.260402,0.212025,0.233745,0.286196
4,"[газовый, упор, капот, geely, coolray, джить, ...",911,0.190871,0.602967,0.412625,0.43003,0.369292,0.227309,0.281212,0.338599,...,0.382774,0.293269,0.130879,0.498813,0.448146,0.419972,0.309378,0.287808,0.240551,0.24168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,"[колбаса, варёный, ближний, горка, молочный, г...",5336,0.229818,0.372594,0.232206,0.334213,0.378298,0.297389,0.226417,0.26116,...,0.370268,0.122907,0.296103,0.191943,0.205788,0.361119,0.482426,0.263724,0.268606,0.221645
299996,"[мёд, берест, прополис, г]",5336,0.170184,0.352033,0.210745,0.317725,0.372194,0.147496,0.195475,0.149831,...,0.357057,0.202648,0.213824,0.144897,0.269203,0.309512,0.463475,0.242953,0.249167,0.254161
299997,"[чай, чёрный, greenfield, magic, yunnan, пакет...",5336,0.386609,0.402984,0.23649,0.266946,0.268559,0.215782,0.176831,0.231559,...,0.305821,0.220966,0.297035,0.202968,0.275578,0.307346,0.345597,0.317516,0.184949,0.296471
299998,"[колбаса, останкино, сервелат, зернистый, вари...",5336,0.202317,0.419306,0.231304,0.346024,0.447397,0.22787,0.239507,0.331926,...,0.414724,0.186084,0.228734,0.261193,0.186017,0.341204,0.519024,0.285845,0.29198,0.216936


In [34]:
model.save('word2vec')
df2.to_csv('final_dataset.csv')

In [9]:
model = Word2Vec.load('word2vec')
df2 = pd.read_csv('../final_dataset.csv', index_col='Unnamed: 0')

In [10]:
df2

,Answer,Name,Товары для дома,Entertainment certificates,Автотовары,ТВ и аудио,Свежие продукты питания,Товары для мам и детей,Одежда,DIY,...,Книги,Красота и здоровье,NewBorn,Спорт и отдых,Ювелирные изделия,Компьютеры и ноутбуки,Продукты питания,Товары для животных,Мебель,Товары повседневного спроса
0,"['оперативный', 'память', 'apacer', 'nox', 'ah...",911,0.233985,0.344255,0.301514,0.469033,0.249215,0.358600,0.227517,0.224213,...,0.435718,0.306469,0.258979,0.242221,0.305348,0.476621,0.154391,0.359499,0.294874,0.344474
1,"['видеокарта', 'kfa2', 'geforce', 'rtx', 'x', ...",911,0.314530,0.521324,0.270000,0.299432,0.286479,0.230729,0.339655,0.357172,...,0.400982,0.335735,0.232374,0.315515,0.425404,0.545673,0.294164,0.365769,0.411719,0.258101
2,"['оперативный', 'память', 'kingston', 'fury', ...",911,0.230929,0.381401,0.299406,0.443391,0.254926,0.357999,0.232324,0.264696,...,0.441260,0.316416,0.266160,0.256677,0.308540,0.458889,0.178846,0.374134,0.299524,0.347242
3,"['свеча', 'зажигание', 'ngk', 'cr7hsa', 'шт']",911,0.277128,0.370164,0.334299,0.427887,0.192633,0.254990,0.137520,0.284665,...,0.401980,0.240009,0.227765,0.203162,0.378024,0.295506,0.260402,0.212025,0.233745,0.286196
4,"['газовый', 'упор', 'капот', 'geely', 'coolray...",911,0.190871,0.602967,0.412625,0.430030,0.369292,0.227309,0.281212,0.338599,...,0.382774,0.293269,0.130879,0.498813,0.448146,0.419972,0.309378,0.287808,0.240551,0.241680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,"['колбаса', 'варёный', 'ближний', 'горка', 'мо...",5336,0.229818,0.372594,0.232206,0.334213,0.378298,0.297389,0.226417,0.261160,...,0.370268,0.122907,0.296103,0.191943,0.205788,0.361119,0.482426,0.263724,0.268606,0.221645
299996,"['мёд', 'берест', 'прополис', 'г']",5336,0.170184,0.352033,0.210745,0.317725,0.372194,0.147496,0.195475,0.149831,...,0.357057,0.202648,0.213824,0.144897,0.269203,0.309512,0.463475,0.242953,0.249167,0.254161
299997,"['чай', 'чёрный', 'greenfield', 'magic', 'yunn...",5336,0.386609,0.402984,0.236490,0.266946,0.268559,0.215782,0.176831,0.231559,...,0.305821,0.220966,0.297035,0.202968,0.275578,0.307346,0.345597,0.317516,0.184949,0.296471
299998,"['колбаса', 'останкино', 'сервелат', 'зернисты...",5336,0.202317,0.419306,0.231304,0.346024,0.447397,0.227870,0.239507,0.331926,...,0.414724,0.186084,0.228734,0.261193,0.186017,0.341204,0.519024,0.285845,0.291980,0.216936


In [24]:
answerList_for_id

Name
0       [Технологии проектирования баз данных, Уголовн...
1       [Крышка dennerle nano cube 30 стеклянная бесцв...
2       [Python для финансистов, Тиктокеры и не только...
3       [Свеча зажигания ngk 2667 bkr7eix 1 шт., Након...
4       [Vbs 1012_щеткодержатель стартера! audi a4/a6/...
                              ...                        
9967    [Интернет-шопинг. реальный путеводитель по вир...
9968    [Rogue company, Nero, The great war: western f...
9969    [Фильтр салонный marshall mc1914 для lada vest...
9970    [--- дневные ходовые огни 8 led, 12 в, Фара пр...
9971    [Грунт dennerle crystal quartz gravel, светло-...
Name: Answer, Length: 9972, dtype: object

In [11]:
df2 = df2.groupby('Name').agg(sum)

In [18]:
df2['max1_col'] = df2.iloc[:,1:-1].apply(lambda row: row.nlargest(3).index[0], axis=1)
df2['max2_col'] = df2.iloc[:,1:-1].apply(lambda row: row.nlargest(3).index[1], axis=1)
df2['max3_col'] = df2.iloc[:,1:-2].apply(lambda row: row.nlargest(3).index[2], axis=1)

In [19]:
df2

,Answer,Товары для дома,Entertainment certificates,Автотовары,ТВ и аудио,Свежие продукты питания,Товары для мам и детей,Одежда,DIY,Аксессуары для электроники,...,Спорт и отдых,Ювелирные изделия,Компьютеры и ноутбуки,Продукты питания,Товары для животных,Мебель,Товары повседневного спроса,max1_col,max2_col,max3_col
Name,,,,,,,,,,,,,,,,,,,,,
0,"['технология', 'проектирование', 'база', 'данн...",8.533210,11.434790,8.117991,9.065236,11.738377,7.182072,7.544563,8.202822,7.788225,...,6.982916,9.366097,8.926042,11.896472,7.260799,8.333665,9.492006,Charity,Крупная бытовая техника,Продукты питания
1,"['крышка', 'dennerle', 'nano', 'cube', 'стекля...",8.396680,11.661532,8.182738,9.211610,11.149660,6.931331,7.190888,8.195827,8.078464,...,7.303533,8.482357,9.382180,10.856840,8.101865,7.574771,9.355747,Charity,Entertainment certificates,Крупная бытовая техника
2,"['python', 'финансист']['тиктокер', 'работать'...",7.709215,12.349491,8.519625,8.962449,12.295290,7.477504,8.124704,7.972781,8.150660,...,7.055805,10.055950,9.780042,12.192337,7.998221,8.534497,9.540855,Charity,Крупная бытовая техника,Entertainment certificates
3,"['свеча', 'зажигание', 'ngk', 'bkr7eix', 'шт']...",7.604977,11.358683,7.995968,8.605865,10.638607,6.335551,7.769094,8.155302,7.822242,...,6.905600,10.078779,10.020630,11.173613,7.232440,7.635621,8.822031,Charity,Entertainment certificates,Продукты питания
4,"['vbs', '1012', 'щёткодержатель', 'стартер', '...",8.749717,10.663217,8.723648,8.418528,12.187252,7.297474,7.547172,7.230455,8.090767,...,6.624671,9.018610,9.048412,11.402588,7.173336,8.031636,9.469151,Свежие продукты питания,Крупная бытовая техника,Charity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,"['интернет', 'шопинг', 'реальный', 'путеводите...",7.063229,10.910937,7.616346,8.682252,11.528293,6.987489,8.300938,7.679736,7.746594,...,6.789707,9.367685,10.398778,11.548881,6.897056,7.931075,8.699869,Charity,Крупная бытовая техника,Продукты питания
9968,"['rogue', 'company']['nero']['the', 'great', '...",8.344623,11.409198,7.822716,8.742957,11.212699,6.825341,8.106432,7.873966,7.949610,...,6.610146,9.712481,10.490744,11.784696,7.631590,7.879627,8.858038,Charity,Продукты питания,Entertainment certificates
9969,"['фильтр', 'салонный', 'marshall', 'mc1914', '...",8.990335,10.923041,8.147182,9.243172,11.692322,7.422830,8.535333,8.155081,7.799951,...,7.295083,9.288435,9.001412,12.237369,7.835769,8.378216,9.797840,Charity,Крупная бытовая техника,Продукты питания


In [20]:
df2['PriceCount_for_id'] = PriceCount_for_id.values
df2

,Answer,Товары для дома,Entertainment certificates,Автотовары,ТВ и аудио,Свежие продукты питания,Товары для мам и детей,Одежда,DIY,Аксессуары для электроники,...,Ювелирные изделия,Компьютеры и ноутбуки,Продукты питания,Товары для животных,Мебель,Товары повседневного спроса,max1_col,max2_col,max3_col,PriceCount_for_id
Name,,,,,,,,,,,,,,,,,,,,,
0,"['технология', 'проектирование', 'база', 'данн...",8.533210,11.434790,8.117991,9.065236,11.738377,7.182072,7.544563,8.202822,7.788225,...,9.366097,8.926042,11.896472,7.260799,8.333665,9.492006,Charity,Крупная бытовая техника,Продукты питания,11072
1,"['крышка', 'dennerle', 'nano', 'cube', 'стекля...",8.396680,11.661532,8.182738,9.211610,11.149660,6.931331,7.190888,8.195827,8.078464,...,8.482357,9.382180,10.856840,8.101865,7.574771,9.355747,Charity,Entertainment certificates,Крупная бытовая техника,25740
2,"['python', 'финансист']['тиктокер', 'работать'...",7.709215,12.349491,8.519625,8.962449,12.295290,7.477504,8.124704,7.972781,8.150660,...,10.055950,9.780042,12.192337,7.998221,8.534497,9.540855,Charity,Крупная бытовая техника,Entertainment certificates,19773
3,"['свеча', 'зажигание', 'ngk', 'bkr7eix', 'шт']...",7.604977,11.358683,7.995968,8.605865,10.638607,6.335551,7.769094,8.155302,7.822242,...,10.078779,10.020630,11.173613,7.232440,7.635621,8.822031,Charity,Entertainment certificates,Продукты питания,218323
4,"['vbs', '1012', 'щёткодержатель', 'стартер', '...",8.749717,10.663217,8.723648,8.418528,12.187252,7.297474,7.547172,7.230455,8.090767,...,9.018610,9.048412,11.402588,7.173336,8.031636,9.469151,Свежие продукты питания,Крупная бытовая техника,Charity,17820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,"['интернет', 'шопинг', 'реальный', 'путеводите...",7.063229,10.910937,7.616346,8.682252,11.528293,6.987489,8.300938,7.679736,7.746594,...,9.367685,10.398778,11.548881,6.897056,7.931075,8.699869,Charity,Крупная бытовая техника,Продукты питания,77566
9968,"['rogue', 'company']['nero']['the', 'great', '...",8.344623,11.409198,7.822716,8.742957,11.212699,6.825341,8.106432,7.873966,7.949610,...,9.712481,10.490744,11.784696,7.631590,7.879627,8.858038,Charity,Продукты питания,Entertainment certificates,140822
9969,"['фильтр', 'салонный', 'marshall', 'mc1914', '...",8.990335,10.923041,8.147182,9.243172,11.692322,7.422830,8.535333,8.155081,7.799951,...,9.288435,9.001412,12.237369,7.835769,8.378216,9.797840,Charity,Крупная бытовая техника,Продукты питания,9109


In [21]:
result = df.groupby('Name').apply(lambda group: dict(zip(group['Company_id'], group['Price'])))

# Создание итогового DataFrame
df2['PriceToCompany'] = result

In [22]:
df2.index = labelencoder.inverse_transform(df2.iloc[:, -5:].index)

In [23]:
df2.to_csv('../final_dataset.csv')

In [24]:
df2

,Answer,Товары для дома,Entertainment certificates,Автотовары,ТВ и аудио,Свежие продукты питания,Товары для мам и детей,Одежда,DIY,Аксессуары для электроники,...,Компьютеры и ноутбуки,Продукты питания,Товары для животных,Мебель,Товары повседневного спроса,max1_col,max2_col,max3_col,PriceCount_for_id,PriceToCompany
Абакелия Мукас Робертовна,"['технология', 'проектирование', 'база', 'данн...",8.533210,11.434790,8.117991,9.065236,11.738377,7.182072,7.544563,8.202822,7.788225,...,8.926042,11.896472,7.260799,8.333665,9.492006,Charity,Крупная бытовая техника,Продукты питания,11072,"{1252535439: 989, 1310763416: 498, 1983695308:..."
Абакумов Геннадий Николаевич,"['крышка', 'dennerle', 'nano', 'cube', 'стекля...",8.396680,11.661532,8.182738,9.211610,11.149660,6.931331,7.190888,8.195827,8.078464,...,9.382180,10.856840,8.101865,7.574771,9.355747,Charity,Entertainment certificates,Крупная бытовая техника,25740,"{1221453650: 1930, 1816854351: 1583, 169278206..."
Абакумова Анна Ефимовна,"['python', 'финансист']['тиктокер', 'работать'...",7.709215,12.349491,8.519625,8.962449,12.295290,7.477504,8.124704,7.972781,8.150660,...,9.780042,12.192337,7.998221,8.534497,9.540855,Charity,Крупная бытовая техника,Entertainment certificates,19773,"{1310763416: 2685, 1252535439: 1359, 139734612..."
Абакумова Евгения Александровна,"['свеча', 'зажигание', 'ngk', 'bkr7eix', 'шт']...",7.604977,11.358683,7.995968,8.605865,10.638607,6.335551,7.769094,8.155302,7.822242,...,10.020630,11.173613,7.232440,7.635621,8.822031,Charity,Entertainment certificates,Продукты питания,218323,"{1085769503: 5580, 1197983357: 273, 1597283624..."
Абалкина Мария Егоровна,"['vbs', '1012', 'щёткодержатель', 'стартер', '...",8.749717,10.663217,8.723648,8.418528,12.187252,7.297474,7.547172,7.230455,8.090767,...,9.048412,11.402588,7.173336,8.031636,9.469151,Свежие продукты питания,Крупная бытовая техника,Charity,17820,"{1085769503: 1360, 1197983357: 508, 1252535439..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Яшина Вера Ивановна,"['интернет', 'шопинг', 'реальный', 'путеводите...",7.063229,10.910937,7.616346,8.682252,11.528293,6.987489,8.300938,7.679736,7.746594,...,10.398778,11.548881,6.897056,7.931075,8.699869,Charity,Крупная бытовая техника,Продукты питания,77566,"{1310763416: 1585, 1397346122: 2945, 183868555..."
Яшина Марина Владимировна,"['rogue', 'company']['nero']['the', 'great', '...",8.344623,11.409198,7.822716,8.742957,11.212699,6.825341,8.106432,7.873966,7.949610,...,10.490744,11.784696,7.631590,7.879627,8.858038,Charity,Продукты питания,Entertainment certificates,140822,"{1983695308: 419, 1310320260: 749, 1838685550:..."
Яшина Юлия Александровна,"['фильтр', 'салонный', 'marshall', 'mc1914', '...",8.990335,10.923041,8.147182,9.243172,11.692322,7.422830,8.535333,8.155081,7.799951,...,9.001412,12.237369,7.835769,8.378216,9.797840,Charity,Крупная бытовая техника,Продукты питания,9109,"{1197983357: 912, 1381091878: 515, 1310320260:..."
Яшкин Константин Александрович,"['дневный', 'ходовой', 'огонь', 'led']['фара',...",7.825512,10.933541,8.531915,8.876257,11.787244,6.743842,7.902981,8.041557,8.085363,...,9.771894,10.591951,7.936089,7.709723,8.976969,Charity,Свежие продукты питания,Крупная бытовая техника,79159,"{1197983357: 507, 1838685550: 2850, 1227447308..."
